In [1]:
import collections
import math
import os
import random
import zipfile
import numpy as np
import urllib
import tensorflow as tf

C:\Users\Philip\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    stateinfo = os.stat(filename)
    
    if stateinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(stateinfo.st_size)
        raise Exception(
        'Failed to verify' + filename + '.Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip',31344016)

Found and verified text8.zip


In [4]:
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

words = read_data(filename)
print('Data size', len(words))

Data size 17005207


In [5]:
vocabulary_size = 50000

def build_dataset(words):
    count = [['UNK', -1]]
    #找出前50000個最常出現字詞
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    
    #以頻數做為編號
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
        
    data = list()
    unk_count = 0
    
    #剩下的詞編號統一為0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0
            unk_count +=1
            
        data.append(index)
    
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)

In [6]:
del words
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


In [7]:
data_index = 0

#num_skips   要為每個單詞生成幾個養樣本，最多不能大於2倍skip_window，並且batch_size必須是它的整數倍
#skip_window 表示每個單詞最遠可以跟鄰近幾個詞生成樣本

def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert (batch_size % num_skips == 0)
    assert (num_skips <= 2 * skip_window)
    
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    #span為對某個單詞創建相關樣本時會用到的單詞數量
    span = 2 * skip_window + 1
    #deque為雙向隊列，這裡我們每次append的時候都只會添加span個變量
    buffer = collections.deque(maxlen=span)
    
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
        
    for i in range(batch_size // num_skips):
        target = skip_window
        targets_to_avoid = [skip_window]
        
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

In [8]:
data_index = 0
batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

3081 originated -> 5234 anarchism
3081 originated -> 12 as
12 as -> 6 a
12 as -> 3081 originated
6 a -> 12 as
6 a -> 195 term
195 term -> 6 a
195 term -> 2 of


In [9]:
batch_size = 128
embedding_size = 128
skip_window = 1
num_skips = 2

valid_size = 16            #用來驗證的單詞數
valid_window = 100         #只從頻數前100高的詞抽取
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64           #噪聲單詞數量

In [13]:
graph = tf.Graph()
with graph.as_default():
    
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)
    
    
    nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
    
    loss = tf.reduce_mean(tf.nn.nce_loss(weights = nce_weights, 
                                         biases = nce_biases,
                                         labels = train_labels,
                                         inputs = embed,
                                         num_sampled = num_sampled,
                                         num_classes = vocabulary_size))
    
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
    
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))   #L2范數
    normalized_embeddings = embeddings / norm    # 標準化
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    
    init = tf.global_variables_initializer()
    
    num_steps = 100001

    with tf.Session(graph=graph) as session:
      # We must initialize all variables before we use them.
        init.run()
        print("Initialized")

        average_loss = 0
        for step in range(num_steps):
            batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
            
            feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

            # We perform one update step by evaluating the optimizer op (including it
            # in the list of returned values for session.run()
            _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
            average_loss += loss_val
        
            if step % 2000 == 0:
                if step > 0:
                    average_loss /= 2000
                    # The average loss is an estimate of the loss over the last 2000 batches.
                    print("Average loss at step ", step, ": ", average_loss)
                    average_loss = 0
                
          

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
            if step % 10000 == 0:
                sim = similarity.eval()
            
                for i in range(valid_size):
                    valid_word = reverse_dictionary[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log_str = "Nearest to %s:" % valid_word
                    
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," % (log_str, close_word)
                print(log_str)
                
        final_embeddings = normalized_embeddings.eval()

Initialized
Nearest to over: eleanor, holbein, henrique, sixth, plate, edgerton, walther, realizing,
Average loss at step  2000 :  113.3340799369812
Average loss at step  4000 :  51.973287120103834
Average loss at step  6000 :  34.75032259297371
Average loss at step  8000 :  23.441740251541137
Average loss at step  10000 :  17.816848940730093
Nearest to over: plate, denver, mike, sixth, determines, roi, seven, tail,
Average loss at step  12000 :  14.021331929922104
Average loss at step  14000 :  11.625479837417602
Average loss at step  16000 :  9.880560529112817
Average loss at step  18000 :  8.978321503996849
Average loss at step  20000 :  7.378712332963944
Nearest to over: anoa, plate, seven, four, recitative, denver, electricity, cfc,
Average loss at step  22000 :  7.433821877241135
Average loss at step  24000 :  6.848018287181854
Average loss at step  26000 :  6.637850007534027
Average loss at step  28000 :  6.190489963889122
Average loss at step  30000 :  5.884298511147499
Nearest

In [14]:
def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    plt.figure(figsize=(18,18))
    
    for i,label in enumerate(labels):
        x, y = low_dim_embs[i,:]
        plt.scatter(x, y)
        plt.annotate(label,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha = 'right',
                     va='bottom')
    plt.savefig(filename)

In [19]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)

plot_only = 100
low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only,:])
labels = [reverse_dictionary[i] for i in range(plot_only)]
plot_with_labels(low_dim_embs, labels)